# This notebook is created to work on the IBM data science capstone project

## Introduction

Introduction where you discuss the business problem and who would be interested in this project.

Data Science teqchniques in Syria has not been deployed but for the rare occasion of some humanitarian reasons. In my project I wanted to explore the neighborhoods in 3 Syiran cities(Damascus, Tartous, Latakia) and find out the similarities between them. In doing so, it would be beneficial for any businesses that currently operate in one city and want to expand to another. Of course it is not only limited to that, Clustering the neighborhood is of high advantage to all kind of market segmentation whether it be retail, services or housing.

## Data

Necessery data for the analysis are the geodata of the Syrian cities and neighborhoods including the coordinates of each. It was not an easy job to find it, but luckily I was able to extract it from a database of the https://data.humdata.org "Administrative Boundaries, Populated Places" that was originaly made for humanitarian studies but works just fine for my project.
After cleaning the data and extracting the neighborhoods of the 3 cities, I will be deploying the Foursquare API to get venues information about the nearby locations

In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
url = 'https://data.humdata.org/dataset/356a63e9-90aa-4b9c-a938-58ef24469c00/resource/82f8d33c-4dcc-46f4-aab5-302fec004714/download/syr_humanitarian_locations_20181219.xlsx'

In [4]:
df = pd.read_excel(url,  sheetname=3, header=0)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [5]:
df.head()

,admin0Name_en,admin0Name_ar,admin0Pcode,admin1Name_en,admin1Name_ar,admin1Pcode,admin2Name_en,admin2Name_ar,admin2Pcode,admin3Name_en,...,admin4Name_en,admin4Name_ar,admin4Pcode,admin4RefName_en,admin4RefName_ar,Location_Pcode,location_type,LastUpdateDate,Latitude_y,Longitude_x
0,Syrian Arab Republic,الجمهورية العربية السورية,SY,Damascus,دمشق,SY01,Damascus,دمشق,SY0100,Damascus,...,Damascus,دمشق,C1001,Damascus,دمشق,C1001,Community,19/12/2018,33.517175,36.276710
1,Syrian Arab Republic,الجمهورية العربية السورية,SY,Damascus,دمشق,SY01,Damascus,دمشق,SY0100,Damascus,...,Yarmuk,اليرموك,C1002,Yarmuk (Damascus),اليرموك - دمشق,C1002,Community,19/12/2018,33.471206,36.305668
2,Syrian Arab Republic,الجمهورية العربية السورية,SY,Aleppo,حلب,SY02,Jebel Saman,جبل سمعان,SY0200,Jebel Saman,...,Musallamiyeh,المسلمية,C1003,Musallamiyeh,المسلمية,C1003,Community,19/12/2018,36.310003,37.205732
3,Syrian Arab Republic,الجمهورية العربية السورية,SY,Aleppo,حلب,SY02,Jebel Saman,جبل سمعان,SY0200,Jebel Saman,...,Esan,عسان,C1004,Esan,عسان,C1004,Community,19/12/2018,36.086354,37.225272
4,Syrian Arab Republic,الجمهورية العربية السورية,SY,Aleppo,حلب,SY02,Jebel Saman,جبل سمعان,SY0200,Jebel Saman,...,Thabiyeh,الذهبية,C1005,Thabiyeh,الذهبية - مركز جبل سمعان,C1005,Community,19/12/2018,36.136870,37.201471


In [6]:
df.columns

Index(['admin0Name_en', 'admin0Name_ar', 'admin0Pcode', 'admin1Name_en',
       'admin1Name_ar', 'admin1Pcode', 'admin2Name_en', 'admin2Name_ar',
       'admin2Pcode', 'admin3Name_en', 'admin3Name_ar', 'admin3Pcode',
       'admin4Name_en', 'admin4Name_ar', 'admin4Pcode', 'admin4RefName_en',
       'admin4RefName_ar', 'Location_Pcode', 'location_type', 'LastUpdateDate',
       'Latitude_y', 'Longitude_x'],
      dtype='object')

### cleaning the data and dropping unuseful features and duplicates in Arabic language

In [7]:
df = df.drop([ 'admin0Name_ar','admin1Name_ar','admin2Name_ar','admin3Name_ar','admin4Name_ar','admin4RefName_ar','admin0Name_en','admin0Pcode','admin1Pcode','admin2Name_en','admin2Pcode'], axis = 1)

In [8]:
df = df.drop(['admin4Name_en','admin4Pcode','Location_Pcode','LastUpdateDate'], axis = 1)

### checking the data of the wanted cities

In [9]:
df.groupby(['admin1Name_en']).size()

admin1Name_en
Al-Hasakeh        1286
Aleppo            1926
Ar-Raqqa           648
As-Sweida          150
Damascus           102
Dar'a              191
Deir-ez-Zor        208
Hama               679
Homs               582
Idleb              614
Lattakia           487
Quneitra            62
Rural Damascus     260
Tartous            471
dtype: int64

In [10]:
#extracting the data of the 3 cities ignoring the suburbs of each
df_syr = df[df['admin3Name_en'].isin(['Damascus','Lattakia','Tartous'])]

In [11]:
df_syr.head()

,admin1Name_en,admin3Name_en,admin3Pcode,admin4RefName_en,location_type,Latitude_y,Longitude_x
0,Damascus,Damascus,SY010000,Damascus,Community,33.517175,36.276710
1,Damascus,Damascus,SY010000,Yarmuk (Damascus),Community,33.471206,36.305668
2431,Lattakia,Lattakia,SY060000,Stamarkho,Community,35.587201,35.850598
2432,Lattakia,Lattakia,SY060000,Qanjara,Community,35.580483,35.810003
2433,Lattakia,Lattakia,SY060000,Maghrit,Community,35.605236,35.831033


In [12]:
df_syr.shape

(185, 7)

In [13]:
df_syr.groupby(['location_type']).size()

location_type
Community       59
Neibourhood    126
dtype: int64

In [14]:
#data contains communities and neighbourhoods, let us get only the neighbourhoods
df_sy = df_syr.loc[df_syr['location_type'] == 'Neibourhood']

In [15]:
df_sy.groupby(['admin3Name_en']).size()

admin3Name_en
Damascus    100
Lattakia     17
Tartous       9
dtype: int64

### The extracted neighbourhoods count are just fine, considering that Damascus is way bigger than Lattakia and Tartous
### now let us visualize a bit

In [16]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [17]:
import folium
latitude=34.8021 #middleeast
longitude=38.9968 #middleeast
map_sy = folium.Map(location = [latitude, longitude], zoom_start = 7)

for lat, long in zip(df_sy['Latitude_y'], df_sy['Longitude_x']):
    folium.CircleMarker(
        [lat, long],
        radius = 3, 
        color = 'blue',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7).add_to(map_sy)
        
map_sy

In [18]:
#defining Foursquare API
CLIENT_ID = 'VQL2BLC2FRVS5PJYKLZPKHE34X0WIGOWYBIPNYWMRMYA45NN'
CLIENT_SECRET = 'EZJVIV2QM30DKCBVMCY5ED0UNQUGKX50M00XS1CFJVD3IJKC'
VERSION = '20190605'

In [19]:
#Droping the location type feature since we only extracted neighbourhoods
df_sy = df_sy.drop('location_type', axis = 1)

In [20]:
df_sy = df_sy.drop(['admin1Name_en','admin3Pcode'], axis = 1)

In [21]:
df_sy.shape

(126, 4)

In [22]:
#Renaming to meaningful feature names
df_sy= df_sy.rename(columns={'admin3Name_en': 'city', 'admin4RefName_en': 'neighbourhood', 'Latitude_y': 'latitude', 'Longitude_x': 'longitude'})


In [23]:
df_sy.tail(10)

,city,neighbourhood,latitude,longitude
7548,Lattakia,Sleiba,35.507859,35.771814
7604,Tartous,Al-Rabieh,34.902691,35.894547
7605,Tartous,Al-Malaab,34.899768,35.882978
7606,Tartous,Al-Mina,34.898970,35.877595
7607,Tartous,Al-Karamah,34.889765,35.879694
7608,Tartous,Al-Hamrat,34.887848,35.886451
7609,Tartous,Al-Mahattah,34.885114,35.901105
7610,Tartous,Arraml,34.881335,35.883974
7611,Tartous,Al-Ghamkah,34.873771,35.889815
7612,Tartous,Southern(Al-rradar),34.859534,35.905418


# Methodology

The aim of the project is to cluster the neighborhoods of the Syrian cities, I will be using the above data to get nearby venues of each neighbourhood utilizing the Foursquare API. The goal is to define characteristics for each of them which will be then useful for the Clustering algorithm to work, I will be using the Kmeans Clustering Algorithm for this purpose and depend on folium maps as I move in with the work.

In [24]:
#Utilizing the foursquare API to get nearby venues to one of the neighbourhoods
LIMIT = 100
radius = 2000
lati = 34.902691 #Al-Rabieh Neighbourhood in Tartus
longi = 35.894547
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lati, 
    longi, 
    VERSION, 
    radius, 
    LIMIT)

In [25]:
results = requests.get(url).json()

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Wan cafe & rest,Café,34.893259,35.887444
1,Al Bassel Public Park,Park,34.892713,35.883975
2,Tartous Port,Boat or Ferry,34.900108,35.875614
3,Barcode,Bar,34.888618,35.883063


#### we can see that there aren't much venues around, and this is in fact a limitation of the project as the Foursquare API does not have enough data of Syrian Venues
### we can still work with that, lets repeat and get nearby venues for all neighbourhoods

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
sy_venues = getNearbyVenues(names=df_sy['neighbourhood'],
                                   latitudes=df_sy['latitude'],
                                   longitudes=df_sy['longitude']
                                  )

Al Hurriya_ Ish Al Werwer
Barza Al Balad
Al Manara
Tishrine
Al Abbas
Asad Addein
Qaboun
Masakin Barza
Naqshabandi
Dummar Sharqiya
Al Masani'
Al Woroud
Ayoubiya
Qasyoun
Nuzha
Al Fayhaa
Abou Jarash
Faris Al Khouri
Sheikh Muhie Addin
Al Arin
Al Madaris
Shoura
Mastaba
Dahiet Dummar
Joubar Sharqi
Al Qusur
Al Marabit
Al Mazra'a
Al 'Adwi
Joubar Gharbi
Rawda
Ma'amouniyyeh
Shuhada
Al Qusa'
Abou Rummaneh
Al Maliki
Haboubi
Masjid Aqsab
Oqaiba
Saroujah
Amara Baraniyyeh
Joura
Amara Jowaniyyeh
Dummar Gharbiya
Hameidiyyeh
Hijaz
Rabwa
Al Qeimariyyeh
Bab Touma
Hariqa
Al Istiqlal
Al Qanawat
Mazzeh 86
Shaghour Jowani
Ma'azanet Ash Shahm
Mazzeh Al Qadimeh
Amin
Baramkeh
Bab Sreijeh
Bab Al Jabiyeh
Bab Sharqi
Ansari
Shaghour Barani
Mazzej Jabal
Qabr Atikah
Suweiqa
Ibn Asaker
Nidal
Al Moujtahed
Kafr Souseh Al Balad
Jalaa
Al Ikhlas
Mazzeh Basatin
Wihdeh
Bilal
Fealat Gharbiyeh
Al Waha
Midan Wastani
Rawdet Al Midan
Zahreh
Haqleh
Bab Maser
Zohour
Daqaq
Sayyida Aisha
Fealat Sharqiyeh
Fardos
Soumariyyeh
Qa'a
Lwan
D

In [30]:
sy_venues.shape

(999, 7)

In [31]:
sy_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Al Hurriya_ Ish Al Werwer,33.563867,36.327223,Toledo Cafe,33.555140,36.336106,Café
1,Al Hurriya_ Ish Al Werwer,33.563867,36.327223,Al-Rabwe,33.553812,36.320340,Nature Preserve
2,Al Hurriya_ Ish Al Werwer,33.563867,36.327223,Alfayha'a Club New Cham,33.551463,36.320434,Sports Club
3,Al Hurriya_ Ish Al Werwer,33.563867,36.327223,Fitness House,33.548083,36.317668,Gym
4,Barza Al Balad,33.558486,36.320459,Al-Rabwe,33.553812,36.320340,Nature Preserve


In [32]:
#let us see how many nearby venues each neighbourhood has
sy_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
7 Nisan,4,4,4,4,4,4
Abou Jarash,12,12,12,12,12,12
Abou Rummaneh,17,17,17,17,17,17
Al 'Adwi,18,18,18,18,18,18
Al Abbas,4,4,4,4,4,4
Al Arin,10,10,10,10,10,10
Al Asad,2,2,2,2,2,2
Al Assali,3,3,3,3,3,3
Al Fayhaa,6,6,6,6,6,6


In [33]:
print('There are {} uniques categories.'.format(len(sy_venues['Venue Category'].unique())))

There are 59 uniques categories.


In [34]:
# one hot encoding
sy_onehot = pd.get_dummies(sy_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sy_onehot['Neighborhood'] = sy_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sy_onehot.columns[-1]] + list(sy_onehot.columns[:-1])
sy_onehot = sy_onehot[fixed_columns]

sy_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Athletics & Sports,Bakery,Bar,Beach,Boat or Ferry,Burger Joint,Bus Station,...,Restaurant,River,Sandwich Place,Shoe Repair,Shop & Service,Shopping Mall,Shopping Plaza,Sports Club,Syrian Restaurant,Waterfront
0,Al Hurriya_ Ish Al Werwer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Al Hurriya_ Ish Al Werwer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Al Hurriya_ Ish Al Werwer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Al Hurriya_ Ish Al Werwer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barza Al Balad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
sy_onehot.shape

(999, 60)

In [36]:
#grouping with the neighbourhood and getting the frequency of venues in each neighborhood
sy_grouped = sy_onehot.groupby('Neighborhood').mean().reset_index()
sy_grouped

,Neighborhood,American Restaurant,Art Gallery,Athletics & Sports,Bakery,Bar,Beach,Boat or Ferry,Burger Joint,Bus Station,...,Restaurant,River,Sandwich Place,Shoe Repair,Shop & Service,Shopping Mall,Shopping Plaza,Sports Club,Syrian Restaurant,Waterfront
0,7 Nisan,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00
1,Abou Jarash,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00
2,Abou Rummaneh,0.058824,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.058824,0.000000,0.00,0.00,0.00
3,Al 'Adwi,0.000000,0.0,0.0,0.0,0.055556,0.0,0.0,0.000000,0.000000,...,0.055556,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00
4,Al Abbas,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.250000,0.000000,0.00,0.25,0.00
5,Al Arin,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.100000,0.000000,...,0.300000,0.000000,0.000000,0.0,0.00,0.000000,0.100000,0.00,0.00,0.00
6,Al Asad,0.000000,0.0,0.0,0.0,0.000000,0.5,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.50,0.000000,0.000000,0.00,0.00,0.00
7,Al Assali,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.333333,0.000000,0.00,0.00,0.00
8,Al Fayhaa,0.000000,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00
9,Al Hurriya_ Ish Al Werwer,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.25,0.00,0.00


In [37]:
sy_grouped.shape

(119, 60)

In [38]:
#exploring the top 5 venues categories of each neighbourhood 
num_top_venues = 5

for hood in sy_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sy_grouped[sy_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----7 Nisan----
         venue  freq
0  Pizza Place  0.25
1         Pier  0.25
2         Café  0.25
3     Pharmacy  0.25
4       Lounge  0.00


----Abou Jarash----
                venue  freq
0                Café  0.50
1               Hotel  0.25
2           Juice Bar  0.08
3  Italian Restaurant  0.08
4        Dessert Shop  0.08


----Abou Rummaneh----
                 venue  freq
0                 Café  0.35
1                Hotel  0.24
2  American Restaurant  0.06
3   Italian Restaurant  0.06
4   Miscellaneous Shop  0.06


----Al 'Adwi----
           venue  freq
0           Café  0.33
1          Hotel  0.11
2          Plaza  0.06
3  Historic Site  0.06
4    Coffee Shop  0.06


----Al Abbas----
                 venue  freq
0    Syrian Restaurant  0.25
1        Shopping Mall  0.25
2                  Gym  0.25
3      Nature Preserve  0.25
4  American Restaurant  0.00


----Al Arin----
                     venue  freq
0               Restaurant   0.3
1                     Café   0.2
2  

                venue  freq
0                Café  0.33
1               Hotel  0.17
2           Juice Bar  0.06
3  Miscellaneous Shop  0.06
4               Plaza  0.06


----Bab Touma----
                venue  freq
0       Historic Site  0.14
1  Miscellaneous Shop  0.14
2           Multiplex  0.14
3          Restaurant  0.14
4               Plaza  0.14


----Baramkeh----
                 venue  freq
0                 Café  0.35
1                Hotel  0.20
2  American Restaurant  0.10
3        Shopping Mall  0.10
4   Italian Restaurant  0.05


----Barza Al Balad----
                 venue  freq
0    Syrian Restaurant  0.25
1        Shopping Mall  0.25
2                  Gym  0.25
3      Nature Preserve  0.25
4  American Restaurant  0.00


----Bilal----
                venue  freq
0       Historic Site   0.2
1          Restaurant   0.2
2               Plaza   0.2
3         Coffee Shop   0.2
4  Persian Restaurant   0.2


----Dahadeel----
                       venue  freq
0  Middle East

                 venue  freq
0                  Bar  0.25
1        Shopping Mall  0.25
2                 Café  0.25
3                  Gym  0.25
4  American Restaurant  0.00


----Oqaiba----
                venue  freq
0                Café  0.30
1               Hotel  0.20
2           Multiplex  0.05
3          Restaurant  0.05
4  Persian Restaurant  0.05


----Oweina----
                 venue  freq
0          Pizza Place  0.25
1                 Café  0.25
2             Pharmacy  0.25
3          Coffee Shop  0.25
4  American Restaurant  0.00


----Qa'a----
                       venue  freq
0  Middle Eastern Restaurant   0.4
1              Movie Theater   0.2
2             Sandwich Place   0.2
3                       Café   0.2
4        American Restaurant   0.0


----Qaboun----
                 venue  freq
0          Bus Station   0.5
1                 Café   0.5
2  American Restaurant   0.0
3          Pizza Place   0.0
4               Market   0.0


----Qabr Atikah----
            

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sy_grouped['Neighborhood']

for ind in np.arange(sy_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sy_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,7 Nisan,Pharmacy,Pier,Pizza Place,Café,Dessert Shop,Comfort Food Restaurant,Construction & Landscaping,Creperie,Eastern European Restaurant,Clothing Store
1,Abou Jarash,Café,Hotel,Italian Restaurant,Juice Bar,Dessert Shop,Comfort Food Restaurant,Historic Site,Gym / Fitness Center,Gym,Grocery Store
2,Abou Rummaneh,Café,Hotel,American Restaurant,Miscellaneous Shop,Shopping Mall,Italian Restaurant,Juice Bar,Multiplex,Dessert Shop,Creperie
3,Al 'Adwi,Café,Hotel,Historic Site,Coffee Shop,Bar,Miscellaneous Shop,Italian Restaurant,Juice Bar,Restaurant,Multiplex
4,Al Abbas,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm


In [41]:
from sklearn.cluster import KMeans

In [42]:
# set number of clusters
kclusters = 5

sy_grouped_clustering = sy_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sy_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 4, 1, 1, 4, 3, 4], dtype=int32)

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sy_merged = df_sy

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sy_merged = sy_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighbourhood')

sy_merged.head() # check the last columns!

,city,neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7252,Damascus,Al Hurriya_ Ish Al Werwer,33.563867,36.327223,4.0,Sports Club,Gym,Café,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7253,Damascus,Barza Al Balad,33.558486,36.320459,4.0,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7254,Damascus,Al Manara,33.555973,36.304052,4.0,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7255,Damascus,Tishrine,33.552449,36.337237,1.0,Dessert Shop,Café,Cafeteria,Gym,Grocery Store,Fast Food Restaurant,Market,Nature Preserve,Bus Station,Recreation Center
7256,Damascus,Al Abbas,33.549085,36.319007,4.0,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm


In [46]:
#ohps we have some NaNs, they did not make the cut to the clusters. I am just going to drop them
sy_merged['Cluster Labels'].isnull

<bound method Series.isnull of 7252    4.0
7253    4.0
7254    4.0
7255    1.0
7256    4.0
7257    4.0
7258    3.0
7259    4.0
7260    4.0
7261    NaN
7262    3.0
7263    1.0
7264    4.0
7265    3.0
7266    4.0
7267    3.0
7268    3.0
7269    3.0
7270    3.0
7271    1.0
7272    3.0
7273    3.0
7274    3.0
7275    1.0
7276    1.0
7277    1.0
7278    3.0
7279    3.0
7280    3.0
7281    3.0
       ... 
7348    4.0
7349    0.0
7350    0.0
7351    0.0
7532    1.0
7533    NaN
7534    NaN
7535    NaN
7536    1.0
7537    3.0
7538    1.0
7539    3.0
7540    1.0
7541    3.0
7542    1.0
7543    1.0
7544    1.0
7545    1.0
7546    3.0
7547    1.0
7548    1.0
7604    3.0
7605    3.0
7606    3.0
7607    3.0
7608    3.0
7609    1.0
7610    3.0
7611    3.0
7612    1.0
Name: Cluster Labels, Length: 126, dtype: float64>

In [47]:
sy_merged = sy_merged.dropna()

In [48]:
#Changing the type of the column from float to int
sy_merged['Cluster Labels'] = sy_merged['Cluster Labels'].apply(np.int64)

# Results

The job is done and the clusters are ready, we can now explore them on folium maps and look at each culster database independently

In [49]:
# create map
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sy_merged['latitude'], sy_merged['longitude'], sy_merged['neighbourhood'], sy_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Folium maps are interactive, you can Zoom in on the cities and explore the clusters

## cluster 1

In [50]:
sy_merged.loc[sy_merged['Cluster Labels'] == 0, sy_merged.columns[[1] + list(range(5, sy_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7343,Tadamon,Sandwich Place,Park,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm,Falafel Restaurant
7347,Qadam Sharqi,Indian Restaurant,Sandwich Place,Park,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm,Falafel Restaurant
7349,El Kermel,Sandwich Place,Movie Theater,Park,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7350,Hettin,Sandwich Place,Movie Theater,Park,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7351,El Faloujeh,Sandwich Place,Movie Theater,Park,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm


## cluster 2

In [51]:
sy_merged.loc[sy_merged['Cluster Labels'] == 1, sy_merged.columns[[1] + list(range(5, sy_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7255,Tishrine,Dessert Shop,Café,Cafeteria,Gym,Grocery Store,Fast Food Restaurant,Market,Nature Preserve,Bus Station,Recreation Center
7263,Al Woroud,Restaurant,River,Middle Eastern Restaurant,Plaza,Burger Joint,Café,Fast Food Restaurant,Shopping Plaza,Farm,Falafel Restaurant
7271,Al Arin,Restaurant,Café,Shopping Plaza,Burger Joint,Plaza,Comfort Food Restaurant,Clothing Store,Gym / Fitness Center,Gym,Grocery Store
7275,Dahiet Dummar,Restaurant,Café,Fast Food Restaurant,Plaza,Burger Joint,Comfort Food Restaurant,Shopping Plaza,Grocery Store,Farm,Coffee Shop
7276,Joubar Sharqi,Bus Station,Waterfront,Coffee Shop,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm,Falafel Restaurant
7277,Al Qusur,Persian Restaurant,Coffee Shop,Plaza,Miscellaneous Shop,Restaurant,Historic Site,Juice Bar,Farm,Falafel Restaurant,Fast Food Restaurant
7283,Ma'amouniyyeh,Historic Site,Restaurant,Coffee Shop,Plaza,Persian Restaurant,Fast Food Restaurant,Farm,Falafel Restaurant,Clothing Store,Grocery Store
7285,Al Qusa',Persian Restaurant,Restaurant,Historic Site,Plaza,Miscellaneous Shop,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Creperie,Waterfront
7289,Masjid Aqsab,Hotel,Café,Restaurant,Multiplex,Persian Restaurant,Miscellaneous Shop,Plaza,Coffee Shop,Juice Bar,Italian Restaurant
7293,Joura,Persian Restaurant,Restaurant,Hotel,Juice Bar,Miscellaneous Shop,Multiplex,Coffee Shop,Café,Plaza,Historic Site


## cluster 3

In [52]:
sy_merged.loc[sy_merged['Cluster Labels'] == 2, sy_merged.columns[[1] + list(range(5, sy_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7331,Zahreh,Café,Sandwich Place,Middle Eastern Restaurant,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7332,Haqleh,Sandwich Place,Middle Eastern Restaurant,Café,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7333,Bab Maser,Sandwich Place,Middle Eastern Restaurant,Movie Theater,Café,Waterfront,Comfort Food Restaurant,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant
7334,Zohour,Middle Eastern Restaurant,Falafel Restaurant,Café,Cambodian Restaurant,Waterfront,Comfort Food Restaurant,Historic Site,Gym / Fitness Center,Gym,Grocery Store
7335,Daqaq,Sandwich Place,Middle Eastern Restaurant,Movie Theater,Café,Waterfront,Comfort Food Restaurant,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant
7336,Sayyida Aisha,Sandwich Place,Middle Eastern Restaurant,Café,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7340,Qa'a,Middle Eastern Restaurant,Sandwich Place,Movie Theater,Café,Waterfront,Comfort Food Restaurant,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant
7342,Dahadeel,Sandwich Place,Middle Eastern Restaurant,Café,Waterfront,Coffee Shop,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7344,Jouret Shreibati,Sandwich Place,Café,Waterfront,Coffee Shop,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7345,Al Qadam,Indian Restaurant,Sandwich Place,Café,Coffee Shop,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm


## cluster 4

In [53]:
sy_merged.loc[sy_merged['Cluster Labels'] == 3, sy_merged.columns[[1] + list(range(5, sy_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7258,Qaboun,Bus Station,Café,Waterfront,Comfort Food Restaurant,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7262,Al Masani',Bus Station,Café,Waterfront,Comfort Food Restaurant,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm
7265,Qasyoun,Juice Bar,Dessert Shop,Café,Park,Waterfront,Comfort Food Restaurant,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant
7267,Al Fayhaa,Café,Juice Bar,Bar,Hotel,Waterfront,Construction & Landscaping,Historic Site,Gym / Fitness Center,Gym,Grocery Store
7268,Abou Jarash,Café,Hotel,Italian Restaurant,Juice Bar,Dessert Shop,Comfort Food Restaurant,Historic Site,Gym / Fitness Center,Gym,Grocery Store
7269,Faris Al Khouri,American Restaurant,Athletics & Sports,Juice Bar,Bar,Fast Food Restaurant,Café,Construction & Landscaping,Historic Site,Gym / Fitness Center,Gym
7270,Sheikh Muhie Addin,Café,Hotel,American Restaurant,Shopping Mall,Italian Restaurant,Juice Bar,Multiplex,Dessert Shop,Eastern European Restaurant,Creperie
7272,Al Madaris,Café,Hotel,American Restaurant,Shopping Mall,Italian Restaurant,Juice Bar,Multiplex,Dessert Shop,Eastern European Restaurant,Creperie
7273,Shoura,Café,Hotel,Italian Restaurant,Juice Bar,Dessert Shop,American Restaurant,Shopping Mall,Bar,Creperie,Gym / Fitness Center
7274,Mastaba,Café,Hotel,Italian Restaurant,Juice Bar,Dessert Shop,American Restaurant,Shopping Mall,Bar,Creperie,Gym / Fitness Center


## cluster 5

In [54]:
sy_merged.loc[sy_merged['Cluster Labels'] == 4, sy_merged.columns[[1] + list(range(5, sy_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7252,Al Hurriya_ Ish Al Werwer,Sports Club,Gym,Café,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7253,Barza Al Balad,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7254,Al Manara,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7256,Al Abbas,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7257,Asad Addein,Gym,Syrian Restaurant,Shopping Mall,Coffee Shop,Historic Site,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm,Falafel Restaurant
7259,Masakin Barza,Syrian Restaurant,Shopping Mall,Gym,Nature Preserve,Waterfront,Coffee Shop,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7260,Naqshabandi,Gym,Bakery,Café,Waterfront,Comfort Food Restaurant,Historic Site,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Farm
7264,Ayoubiya,Gym,Café,Syrian Restaurant,Shopping Mall,Bakery,Juice Bar,Fast Food Restaurant,Waterfront,Eastern European Restaurant,Creperie
7266,Nuzha,Shopping Mall,Gym,Bar,Café,Waterfront,Comfort Food Restaurant,Historic Site,Gym / Fitness Center,Grocery Store,Fast Food Restaurant
7328,Al Waha,Shopping Mall,American Restaurant,Café,Coffee Shop,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Fast Food Restaurant,Farm


# Discussion

Eventhough there were some limitations in this project, I find that it came up with good results, we can look through each cluster and easily understand what the charctaristics of it are, what we might find there, and what is popular. We can also define what business opportunity it gives out. Along with finding the similar neighborhoods in each city and how it may be utilized in businesses expansions

# Conclusion

Eventhough I knew of the limitation I might face and hardships to get the data, I insisted on doing this Capstone Project on Syrian data. You can search online and find barely a handful of data science projects on Syrian data and they are established for humanitarian studies, and non explored with the aim of business opportunities.
I look forward to any feedback and any collaborations on future projects.